# Analysis of clinical variatns of ORC1 protein

In [7]:
from clinvar_functions import *
import pandas as pd

## ORC1 regions

In [3]:
LLPS = [(360, 382), (412, 476), (577, 636), (677, 729), (757, 816)] # Chinees paper
IDR_total = [(183, 476)] # Chinees paper
G4_RNA = [(413, 511)] # Hoshina paper
CDCD6=[(180, 240)] # CDC6 paper


In [6]:
G4_RNA_positions = generate_list(G4_RNA)
LLPS_positions = generate_list(LLPS)

## Read data

In [ ]:

# Set option to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv('clinvar_result.csv', sep = "\t")
df = df[df['Gene(s)'] == 'ORC1'] # save only ORC1 gene - delete 7 records
df.shape

In [ ]:
df.head()

In [22]:
# Subset of data: only mutations in protein
df_proteins = df[df['Protein change'].apply(lambda x: isinstance(x, str))]

## Look at the different columns of df

In [ ]:
df["Condition(s)"].value_counts()

In [ ]:
df["Germline classification"].value_counts()

In [ ]:
df['Germline review status'].value_counts()

## Get list of transcript IDs

In [ ]:
IDs = df['Name'].apply(lambda x: x.split(":")[0])
IDs.value_counts()

## Rename Condition

In [ ]:
df['Condition(s)'].unique()

In [29]:
df['Condition_new'] = df['Condition(s)'].map(rename_condition, na_action='ignore')

In [ ]:
df['Condition_new'].value_counts()

## Groupby

In [ ]:
df.groupby("Germline review status")["Condition_new"].value_counts()

## Subsets

In [29]:
G4_RNA_positions

In [87]:
# Protein change: criteria provided, multiple submitters, no conflicts + Meier-Gorlin syndrome 1 probaly
protein_mut = df['Protein change'][(df['Germline review status'] == 'criteria provided, multiple submitters, no conflicts') & (df['Condition_new']=='not provided')]

In [90]:
# Protein change + criteria provided, single submitter + Meier-Gorlin syndrome 1 probaly
protein_mut = df['Protein change'][(df['Germline review status'] == 'criteria provided, single submitter') & (df['Condition_new']=='not provided')]

In [ ]:
yes, no = mutation_analysis(protein_mut, G4_RNA_positions)  

In [ ]:
yes

In [ ]:
no